# Import Packages and Data

In [1]:
import tensorflow
import keras
import pandas as pd
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
import pickle
from keras.layers import Dense
from keras.models import Sequential
from keras.optimizers import RMSprop, Adadelta, Adam

Using TensorFlow backend.


In [2]:
data = pd.read_csv('../data/buffer_overflow_data.csv.gz')
labels = data.copy()
del labels['Unnamed: 0']
del labels['Unnamed: 0.1']
del labels['filename']
del labels['code']
del labels['flaw']
del labels['flaw_loc']
labels = labels.drop_duplicates().sort_values('testcase_ID').reset_index(drop=True)

In [3]:
x = pd.read_csv("../data/buffer_overflow_graph_embeddings.csv.gz")
x['testcase_ID'] = x['type']
del x['type']
x = x.sort_values("testcase_ID").reset_index(drop=True)

In [4]:
y = labels['bug']
x = x.drop('testcase_ID', axis='columns')

In [5]:
import sklearn

In [6]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.5, random_state = 0)

# [Neural tutorial](https://towardsdatascience.com/neural-networks-from-scratch-easy-vs-hard-b26ddc2e89c7)

In [7]:
with open('../data/baseline-model-binary','rb') as f:
    model = pickle.load(f)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [8]:
model.evaluate(x_test,y_test)

10751/10751 [==============================] - 0s 43us/step


[0.20033853630328355, 0.9128453166371068]

In [9]:
y_pred = model.predict_classes(x)

In [10]:
import seaborn as sns
from sklearn import metrics
import matplotlib

In [11]:
y_predict = model.predict(x_test)

In [12]:
import numpy as np
from matplotlib.colors import LogNorm

Change Confusion Matrix Below

In [13]:
confusion_matrix = pd.DataFrame(
    data=metrics.confusion_matrix((y_test.values+0).argmax(axis=1), y_predict.argmax(axis=1)), 
)
confusion_figure, confusion_axes = matplotlib.pyplot.subplots()
confusion_figure.set_size_inches(15, 12)
confusion_axes.set_title(
    'Confusion matrix showing the frequency of \n'
    'correct and incorrect bug classification predictions.'
    '\n\n'  # hack to avoid overlap with x-axis labels below
)
sns.heatmap(confusion_matrix, norm=LogNorm(confusion_matrix.min().min(),confusion_matrix.max().max()),
            vmin = 0.001, vmax=1000)


AxisError: axis 1 is out of bounds for array of dimension 1

# Random Forest

In [14]:
from sklearn.ensemble import RandomForestRegressor
# Instantiate model with 1000 decision trees
rf = RandomForestRegressor(n_estimators = 1, random_state = 42)
# Train the model on training data
rf.fit(x_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=1, n_jobs=None,
                      oob_score=False, random_state=42, verbose=0,
                      warm_start=False)

In [24]:
rf.score(x_test, y_test)

-0.016853458189709514

In [29]:
y_pred_rf = pd.DataFrame(rf.predict(x_test))

In [30]:
from sklearn.metrics import accuracy_score

In [31]:
accuracy_score(y_test, y_pred_rf)

0.7457910892010046

Sort Visualisation Below!

In [22]:
# from keras.utils import to_categorical

# from visualisation import cve_predict_table

ModuleNotFoundError: No module named 'visualisation'

In [17]:
#example_script = x_test.iloc[0,]
#example_label = y_test.iloc[0,]

In [31]:
#cve_predict_table(data, rf, example_script, example_label)

NameError: name 'keras' is not defined

# KNN

## 1NN

In [32]:
from sklearn.neighbors import KNeighborsClassifier 

knn = KNeighborsClassifier(n_neighbors=1,weights = 'distance')
knn.fit(x_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=1, p=2,
                     weights='distance')

In [33]:
score=knn.score(x_test,y_test)

In [34]:
score

0.8069946981676123

2NN returns the same as one

## 3NN

In [41]:
from sklearn.neighbors import KNeighborsClassifier 

knn = KNeighborsClassifier(n_neighbors=3,weights = 'distance')
knn.fit(x_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=3, p=2,
                     weights='distance')

In [42]:
score=knn.score(x_test,y_test)

In [43]:
score

0.7979722816482188

## Gaussian Naive Bayes

In [44]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
y_pred_gnb = gnb.fit(x_train, y_train).predict(x_test)

In [52]:
print("Number of mislabeled points out of a total %d points : %d"
       % (x_test.shape[0],(y_test != y_pred_gnb).sum()))
print("Accuracy " + str(1- (y_test != y_pred_gnb).sum()/x_test.shape[0]))

Number of mislabeled points out of a total 10751 points : 4348
Accuracy 0.5955725048832667


## SVM



In [56]:
from sklearn import svm
clf_svm = svm.SVC(kernel = 'poly', gamma='scale')
clf_svm.fit(x_train, y_train)  

y_pred_svm = clf_svm.predict(x_test)

In [57]:
print("Number of mislabeled points out of a total %d points : %d"
       % (x_test.shape[0],(y_test != y_pred_svm).sum()))
print("Accuracy" + str(1- (y_test != y_pred_svm).sum()/x_test.shape[0]))

Number of mislabeled points out of a total 10751 points : 1334
Accuracy0.8759185192075156


# Logistic Regression

In [58]:
from sklearn.linear_model import LogisticRegression

In [59]:
logreg = LogisticRegression()
logreg.fit(x_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [60]:
y_pred_lr = logreg.predict(x_test)
print('Accuracy ' + str(logreg.score(x_test, y_test)))

Accuracy 0.8430843642451865


# AdaBoost

In [62]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.datasets import make_classification

In [63]:
clf_ab = AdaBoostClassifier(n_estimators=100, random_state=0)
clf_ab.fit(x_train, y_train)

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None, learning_rate=1.0,
                   n_estimators=100, random_state=0)

In [74]:
y_pred_ab = clf_ab.predict(x_test)
# y_pred = y_pred.reshape(-1,1)
# y_pred.shape

In [75]:
print("Number of mislabeled points out of a total %d points : %d"
       % (x_test.shape[0],(y_test != y_pred_ab).sum()))
print("Accuracy" + str(1- (y_test != y_pred_ab).sum()/x_test.shape[0]))

Number of mislabeled points out of a total 10751 points : 2223
Accuracy0.7932285368802903
